In [3]:
import numpy as np  # Numerical operations library
import zipfile  # To handle zip files
import os  # Operating system interfaces
from PIL import Image  # Image processing library
import tensorflow as tf  # Machine learning and neural network library
from io import BytesIO  # Input/output Streams
from sklearn.svm import OneClassSVM  # One-class SVM from scikit-learn
import joblib  # Library for saving and loading ML models
from google.colab import files  # Google Colab file handling

# Prompt the user to upload a OneClassSVM model file
uploaded = files.upload()
model_filename = next(iter(uploaded))
classifier = joblib.load(model_filename)  # Load the OneClassSVM model

# Prompt the user to upload a ZIP file
uploaded_zip = files.upload()
zip_filename = next(iter(uploaded_zip))  # Get the name of the uploaded ZIP file

# Initialize the MobileNetV2 model for feature extraction
feature_extractor = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet')
feature_extractor = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.GlobalAveragePooling2D()  # Add Global Average Pooling layer
])

# Function: Process and preprocess the image
def process_image(image):
    image = image.resize((224, 224))  # Resize the image to the expected input size
    image_array = np.array(image)  # Convert the image to a numpy array
    image_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # Preprocess the image
    return image_preprocessed

# Function: Extract images from a ZIP file and predict using the classifier
def predict_from_zip(zip_data, feature_extractor, classifier, custom_threshold=0.0):
    predictions = []  # List to store predictions
    with zipfile.ZipFile(BytesIO(zip_data[zip_filename]), 'r') as zip_ref:  # Open the ZIP file
        for file in zip_ref.namelist():  # Loop over the files in the ZIP
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):  # Check for image file extensions
                with zip_ref.open(file) as image_file:  # Open the image file
                    image = Image.open(image_file)  # Load the image
                    image = image.convert('RGB')  # Convert the image to RGB
                    processed_img = process_image(image)  # Preprocess the image
                    feature = feature_extractor.predict(np.expand_dims(processed_img, axis=0)).flatten()  # Extract features

                    # Change: Use the decision function and custom threshold for prediction
                    score = classifier.decision_function([feature])  # Get the decision score for the image
                    prediction = 1 if score > custom_threshold else -1  # Predict based on the threshold

                    predictions.append((file, prediction, score))  # Append the prediction to the list
    return predictions

# You can adjust this threshold to make the model more lenient
custom_threshold = -3.0  # Lower this value to reduce the number of false negatives

# Execute the predictions
predictions = predict_from_zip(uploaded_zip, feature_extractor, classifier, custom_threshold)

# Output the prediction results
for image_name, pred, score in predictions:
    result = 'belongs to the model' if pred == 1 else 'does not belong to the model'  # Determine the result
    print(f"Image {image_name}: {result} (Score: {score})")  # Print the result and score


Saving Mehl-499_model.pkl to Mehl-499_model (1).pkl


Saving Mehl-499-a.zip to Mehl-499-a.zip


1/1 [==============================] - 0s 51ms/step
Image Mehl-499-a/8818_02_03_2021-03-08_15-47-20_albedo.tif: belongs to the model (Score: [5.41668384])
Image Mehl-499-a/8818_02_03_2021-03-08_15-47-20_normal.tif: belongs to the model (Score: [5.18945007])
Image Mehl-499-a/8818_02_04_2021-03-08_15-50-58_albedo.tif: does not belong to the model (Score: [-3.56817668])
Image Mehl-499-a/8818_02_04_2021-03-08_15-50-58_normal.tif: belongs to the model (Score: [3.1626562])
Image Mehl-499-a/8818_02_08_2021-03-11_08-09-14_albedo.tif: belongs to the model (Score: [1.95416465])
Image Mehl-499-a/8818_02_08_2021-03-11_08-09-14_normal.tif: belongs to the model (Score: [4.73715661])
Image Mehl-499-a/8818_02_10_2021-03-09_08-45-55_albedo.tif: belongs to the model (Score: [-1.44331444])
Image Mehl-499-a/8818_02_10_2021-03-09_08-45-55_normal.tif: belongs to the model (Score: [1.74177794])
Image Mehl-499-a/8818_02_12_2021-03-09_08-53-21_albedo.tif: belongs to the model (Score: [2.75890973])
Image Mehl-4